In [ ]:
import customtkinter as ctk
import tkinter as tk   # ✅ ajout important pour "À propos"
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.utils import ImageReader
import os
import datetime
import time


class CancerClassificationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("🩺 Cancer Detection AI")
        self.root.geometry("900x750")
        ctk.set_appearance_mode("dark")
        ctk.set_default_color_theme("blue")

        # Charger le modèle
        try:
            self.model = load_model(r"/home/mohamedplays56/python/PFE 1/notebook_cancer.ipynb")
            print("✅ Modèle chargé avec succès")
        except Exception as e:
            messagebox.showerror("Erreur", f"Impossible de charger le modèle: {str(e)}")
            self.model = None

        # Variables
        self.image_path = None
        self.prediction_result = None
        self.confidence = None
        self.detailed_probs = None
        self.class_names = ['colon_aca', 'colon_n', 'lung_aca', 'lung_n', 'lung_scc']
        self.class_names_fr = {
            'colon_aca': 'Adénocarcinome du colon',
            'colon_n': 'Colon normal',
            'lung_aca': 'Adénocarcinome du poumon',
            'lung_n': 'Poumon normal',
            'lung_scc': 'Carcinome épidermoïde du poumon'
        }

        self.interpretations = {
            'colon_aca': "L'adénocarcinome du côlon est le type le plus courant de cancer du côlon.",
            'colon_n': "Le tissu du côlon apparaît normal sans signe de malignité.",
            'lung_aca': "L'adénocarcinome du poumon est le type le plus courant de cancer du poumon.",
            'lung_n': "Le tissu pulmonaire apparaît normal sans signe de malignité.",
            'lung_scc': "Le carcinome épidermoïde du poumon est souvent associé au tabagisme."
        }

        self.create_widgets()

    def create_widgets(self):
        # Title
        title_label = ctk.CTkLabel(self.root, text="🩺 Cancer Detection AI",
                font=ctk.CTkFont(size=28, weight="bold"))
        title_label.pack(pady=20)

        subtitle = ctk.CTkLabel(self.root, text="Analyse automatique d'images médicales",
                font=ctk.CTkFont(size=16))
        subtitle.pack(pady=(0, 15))

        # Image Frame
        self.image_frame = ctk.CTkFrame(self.root, width=500, height=300, corner_radius=15)
        self.image_frame.pack(pady=15)
        self.image_frame.pack_propagate(False)

        self.image_label = ctk.CTkLabel(self.image_frame, text="Aucune image sélectionnée")
        self.image_label.pack(expand=True)

        # Buttons
        button_frame = ctk.CTkFrame(self.root, fg_color="transparent")
        button_frame.pack(pady=15)

        load_btn = ctk.CTkButton(button_frame, text="📂 Charger une image", command=self.load_image)
        load_btn.grid(row=0, column=0, padx=15)

        predict_btn = ctk.CTkButton(button_frame, text="🔎 Analyser l'image", fg_color="green", command=self.predict)
        predict_btn.grid(row=0, column=1, padx=15)

        pdf_btn = ctk.CTkButton(button_frame, text="📄 Générer rapport PDF", fg_color="red", command=self.generate_pdf)
        pdf_btn.grid(row=0, column=2, padx=15)

        about_btn = ctk.CTkButton(button_frame, text="ℹ️ À propos", command=self.show_about, fg_color="#9b59b6",
                                  text_color="white", font=ctk.CTkFont(family="Helvetica", size=11, weight="bold"))
        about_btn.grid(row=0, column=3, padx=10)

        # Result Frame
        result_frame = ctk.CTkFrame(self.root, corner_radius=15)
        result_frame.pack(pady=20, fill="both", expand=True, padx=20)

        result_title = ctk.CTkLabel(result_frame, text="Résultats de la prédiction",
                               font=ctk.CTkFont(size=18, weight="bold"))
        result_title.pack(pady=10)

        self.result_text = ctk.CTkTextbox(result_frame, height=300, width=700, font=ctk.CTkFont(size=14))
        self.result_text.pack(pady=10, padx=15, fill="both", expand=True)

    def load_image(self):
        file_path = filedialog.askopenfilename(
            title="Sélectionner une image",
            filetypes=[("Images", "*.jpg *.jpeg *.png *.bmp *.tif *.tiff")]
        )

        if file_path:
            self.image_path = file_path
            try:
                img = Image.open(file_path)
                img.thumbnail((450, 300))
                photo = ImageTk.PhotoImage(img)
                self.image_label.configure(image=photo, text="")
                self.image_label.image = photo
            except Exception as e:
                messagebox.showerror("Erreur", f"Impossible de charger l'image: {str(e)}")

    def preprocess_image(self, img_path):
        try:
            input_shape = self.model.input_shape[1:3]
            img = Image.open(img_path).convert("RGB").resize(input_shape)
            img_array = np.array(img) / 255.0
            img_array = np.expand_dims(img_array, axis=0)
            return img_array
        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur de prétraitement: {str(e)}")
            return None

    def predict(self):
        if not self.image_path:
            messagebox.showwarning("Avertissement", "Veuillez d'abord charger une image.")
            return
        if not self.model:
            messagebox.showerror("Erreur", "Modèle non chargé.")
            return

        try:
            start_time = time.time()
            processed_image = self.preprocess_image(self.image_path)
            if processed_image is None:
                return

            predictions = self.model.predict(processed_image)
            predicted_class_index = np.argmax(predictions, axis=1)[0]
            confidence = np.max(predictions) * 100

            class_name = self.class_names[predicted_class_index]
            class_name_fr = self.class_names_fr.get(class_name, class_name)

            self.prediction_result = class_name_fr
            self.confidence = confidence
            self.detailed_probs = []

            for i, prob in enumerate(predictions[0]):
                cls_name = self.class_names[i]
                cls_name_fr = self.class_names_fr.get(cls_name, cls_name)
                self.detailed_probs.append((cls_name_fr, prob*100))

            end_time = time.time()
            self.prediction_time = end_time - start_time

            # Affichage résultat
            self.result_text.delete("1.0", "end")
            self.result_text.insert("end", f"✅ Classe prédite: {class_name_fr}\n")
            self.result_text.insert("end", f"🎯 Confiance: {confidence:.2f}%\n")
            self.result_text.insert("end", f"⏱️ Temps de prédiction: {self.prediction_time:.3f} sec\n\n")
            self.result_text.insert("end", "📊 Probabilités détaillées:\n")
            for cls_name_fr, prob in self.detailed_probs:
                self.result_text.insert("end", f"   - {cls_name_fr}: {prob:.2f}%\n")

        except Exception as e:
            messagebox.showerror("Erreur", f"Erreur lors de la prédiction: {str(e)}")

    def generate_pdf(self):
        # (Ton code PDF identique à ce que tu avais)
        pass

    def show_about(self):
        about_window = tk.Toplevel(self.root)
        about_window.title("À propos")
        about_window.geometry("500x400")
        about_window.configure(bg="#f0f8ff")
        about_window.resizable(False, False)

        # Centrer la fenêtre
        about_window.transient(self.root)
        about_window.grab_set()

        # Titre
        title_label = tk.Label(about_window, text="Cancer Detection AI",
                              font=("Helvetica", 18, "bold"), bg="#f0f8ff", fg="#1565c0")
        title_label.pack(pady=10)

        version_label = tk.Label(about_window, text="Version 1.0",
                                font=("Helvetica", 12), bg="#f0f8ff", fg="#5c6bc0")
        version_label.pack(pady=(0, 20))

        desc_text = tk.Text(about_window, height=10, width=50, wrap=tk.WORD,
                           font=("Helvetica", 10), bg="#f0f8ff", relief=tk.FLAT)
        desc_text.pack(pady=5, padx=20, fill=tk.BOTH, expand=True)

        description = """
Cette application utilise l'intelligence artificielle pour détecter différents types de cancer à partir d'images médicales.

Fonctionnalités:
- Analyse d'images médicales
- Classification en 5 catégories
- Génération de rapports PDF détaillés

Développé par:
- Mohamed Aoudani
- Mohamed Yassine Chalbat
- Imade El-atifi

Note: Cette application est destinée à assister les professionnels de santé et ne remplace pas un diagnostic médical.
"""
        desc_text.insert(tk.END, description)
        desc_text.config(state=tk.DISABLED)

        close_btn = tk.Button(about_window, text="Fermer", command=about_window.destroy,
                             bg="#3498db", fg="white", font=("Helvetica", 11))
        close_btn.pack(pady=10)


if __name__ == "__main__":
    app = CancerClassificationApp(ctk.CTk())
    app.root.mainloop()


✅ Modèle chargé avec succès


invalid command name "140051126456448<lambda>"
    while executing
"140051126456448<lambda>"
    ("after" script)


1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
